#### Description
**Given**: $f(x): \mathbb{R}^d \rightarrow \mathbb{R}$, where $d \geq 1$ (dimension).  
**To find**: $f_N(x): \mathbb{R}^d \rightarrow \mathbb{R}$,
where $N \geq 0$ is the highest order of the approximating multivariate polynomial basis used.


$$f_N(x) = \sum_{|\mathbf{i}|\leq N} \hat{f}_{\mathbf{i}}\Phi_{\mathbf{i}}(x) = \sum_{ip = 1}^P \hat{f}_{ip}\Phi_{ip}(x)$$

In the above expansion, $\mathbf{i}$ is the multi-index notation ($\mathbf{i} = (i_1, i_2,\ldots,i_d) \in \mathbb{N}_0^d$). The central part and the right part of the above equation show the conversion of multi-index notation to a graded lexicographic notation. This has been used interchangeably in the code.

$\hat{f}_\mathbf{i}$ is approximated using the tensor product quadrature (given a quadrature rule for each dimension 'd') as follows:

$$\hat{f}_\mathbf{i} = \hat{f}_{ip} = \sum_{{i_1} = 1}^q\sum_{{i_2} = 1}^q\ldots\sum_{{i_d} = 1}^q f(x_{\mathbf{i}})\big(\Pi_{j = {i_1}}^{i_d}w_j\big) = \sum_{evalPts = 1}^{q^d}f(x_{evalPts})\Pi_{j=1}^dw_j$$

---
## Start of Script

In [22]:
clear; clc; format long;

#### Given below is the function that we want to approximate. Input to the function is a vector x of size (Number of Points * dimension)
##### In this case we have used a dimension of 2 and the function $f(x,y) = xy^2$. This function has been chosen to test the exactness of the quadrature rule. The function that has been commented out is the $C^{\infty}$ oscillatory Genz function. The problem is that I do not get spectral convergence when I approximate this $C^{\infty}$ function.

In [23]:
f = @(x) x(:,1).*x(:,2).^2;%cos(2*pi + 5.0*(x(:,1) + x(:,2)));%x(:,1).*exp(x(:,2))./(1 + x(:,3).^2);
d = 2; %dimension of the random vector

#### Given below is the input for type of orthogonal polynomial basis. Alternatively, use 'Hermite' as the input.

In [24]:
polyBasis = 'Legendre';

#### Given below is the following:
* #### Generation of various Gaussian-Quadrature nodes and weights.  
* #### $\texttt{setprod}$ function takes in an array of points, dimension 'd' and gives the cartesian product as a matrix of size (Length of input vector^d * d)

In [25]:
%Quadrature rule to compute the approximated function coefficients
q = 50;
[xi,w] = gaussQuad(q,polyBasis);
eval_pts = setprod(xi,d); %(q^d points)
weights = setprod(w,d);

%Quadrature rule for the mean square error
Q = 70;
[xi_mse,w_mse] = gaussQuad(Q,polyBasis);
eval_pts_mse = setprod(xi_mse,d); %(Q^d points)
weights_mse = setprod(w_mse,d);

### The function approximation takes place in the cells below
* #### $\texttt{monomialDegrees}$ function takes in the dimension 'd', 'N' the maximum degree of the polynomial basis and gives a matrix of size (P * d), where $P = {N+d\choose N}$. This matrix contains all the combinations of $\mathbf{i} = (i_1,\ldots,i_d)$ such that $||\mathbf{i}||_1 \leq N$ in a graded lexicographing order.
* #### In my code the combinations of the multi-index $\mathbf{i}$ are stored in the variable $\texttt{lexOrdering}$.

In [26]:
order = [0,1,2,3,4,5,6,7,8]; %maximum degree of the multivariate polynomial
MSE = []; %Empty array to store the mean-squared error

#### The 'for' loop below is used to obtain approximate functions of different orders
* $\hat{f}_{\mathbf{i}} \approx \frac{1}{\Pi_{j={i_1}}^{i_d}\gamma_j}\sum_{evalPts = 1}^{q^d} \texttt{legendre}(x_{evalPts}, \mathbf{i})f(x_{evalPts})\Pi_{j=1}^dw_{j}^{evalPts}$
* $\texttt{legendre}$ function takes in a set of points as a matrix of size (Number of eval. points \* d) (generated from the $\texttt{setprod}$ function) and an array $\mathbf{i} = (i_1, \ldots,i_d)$ to give out an array of size (Number of eval. points * 1). The mathematical expression for this function is as follows:  


$$\texttt{legendre}(x,\mathbf{i}) = \Pi_{j=1}^d \phi_{i_j}(x_j),$$
where $x \in \mathbb{R}^d$. In this case, my function can handle vector values as inputs: $[x_1, x_2, \ldots], x_i \in \mathbb{R}^d$.
* The variable $\texttt{gamma}$ contains the normalization constants of the univariate orthogonal polynomials of degree upto 'N'.
* Change the function to $\texttt{hermite}(x_{evalPts}, \mathbf{i})$ and set the variable $\texttt{polyBasis = 'Hermite'}$ above.

In [27]:
for N = order
%Generating the various combinations of 'i' = (i_1, i_2, ... i_d)
    lexOrdering = monomialDegrees(d,N);
%Pre-computing the normalization factors
    if strcmp(polyBasis,'Hermite')
        gamma = factorial(0:N); %Hermite
    else
        gamma = 2.0./(2*(0:N) + 1.0); %Legendre
    end

%Obtaining the number of all the combinations
    P = size(lexOrdering,1);

%Initialization of arrays for the approximated function
    fhat = zeros(P,1);
    fapprox = 0;

   
%The 'for' loop below goes through each of the combinations of 'i'
    for i_P = 1:P
        fhat(i_P,1) = (sum(legendre(eval_pts,lexOrdering(i_P,:)').*f(eval_pts).*prod(weights,2)))/prod(gamma(lexOrdering(i_P,:)+1));
        fapprox = fapprox + fhat(i_P,1)*legendre(eval_pts_mse, lexOrdering(i_P,:)');
    end
    %Computing the mean squared error
    MSE = [MSE;sqrt(sum((fapprox - f(eval_pts_mse)).^2.*prod(weights_mse,2)))];
end
MSE


MSE =

   0.516397779494323
   0.344265186329549
   0.344265186329549
   0.000000000000003
   0.000000000000003
   0.000000000000003
   0.000000000000003
   0.000000000000003
   0.000000000000004



#### This part is just for visualization, works only for d=2

In [28]:
% if d == 2
%     [x,y] = meshgrid(xi,xi);
%     fn = reshape(f(eval_pts),q,q);
%     fn_approx = reshape(fapprox,q,q);
%     surf(x,y,fn,'FaceColor',[1,0,0]); %Exact function is in red
%     hold on;
%     surf(x,y,fn_approx,'FaceColor',[0,1,0]); %Approx. function is in green
%     lim = 2;
%     xlim([-lim,lim]);ylim([-lim,lim]);
% end